<a href="https://colab.research.google.com/github/SingTown/openmv_tensorflow_training_scripts/blob/main/openmv_mobilenet_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train MobilenetV2 and Save to OpenMV

This Code is for TensorFlow 2

## Download Cats vs Dogs dataset from github


In [1]:
!git clone --branch master --depth=1 https://github.com/haritha91/Cats-Dogs-Classifier---Keras.git cat_dog

fatal: destination path 'cat_dog' already exists and is not an empty directory.


In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras, lite

CLASS_NUM = 2 # dog and cat
INPUT_SIZE = 96 #image size is 96 * 96 *3

In [3]:
from keras.preprocessing.image import ImageDataGenerator

def image_to_0_1(img):
    img = img.astype(np.float32) / 255.0
    img = (img - 0.5) * 2
    return img

train_datagen = ImageDataGenerator(preprocessing_function=image_to_0_1,
                    horizontal_flip = True)

test_datagen = ImageDataGenerator(preprocessing_function=image_to_0_1,)

training_set = train_datagen.flow_from_directory('cat_dog/dataset/training_set',
                          target_size = (INPUT_SIZE, INPUT_SIZE),
                          batch_size = 32)

test_set = test_datagen.flow_from_directory('cat_dog/dataset/test_set',
                        target_size = (INPUT_SIZE, INPUT_SIZE),
                        batch_size = 32)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


## Define Model

alpha=0.35 for lower size

Note: Do NOT use GlobalAveragePooling2D Which is NOT supported by OpenMV

In [4]:
base = keras.applications.MobileNetV2(
    include_top=False,
    alpha=0.35,
    weights="imagenet",
    input_shape=(INPUT_SIZE, INPUT_SIZE, 3)
)
base.trainable = False

x = base.output
flatten = keras.layers.Flatten()(x)
dropout = keras.layers.Dropout(0.1)(flatten)
predictions = keras.layers.Dense(CLASS_NUM, activation='softmax')(dropout)
model = keras.models.Model(inputs=base.input, outputs=predictions)
#print(model.summary())

## Train model
After 1 epoches, accuracy is about 91% 

In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(training_set, epochs = 1, validation_data = test_set)


250/250 [==============================] - 84s 303ms/step - loss: 0.4134 - accuracy: 0.8891 - val_loss: 0.3897 - val_accuracy: 0.8960


## Full Integer Quantization

ref: https://www.tensorflow.org/lite/performance/post_training_quantization#full_integer_quantization

In [6]:
quant_set = test_datagen.flow_from_directory('cat_dog/dataset/test_set',
                        target_size = (INPUT_SIZE, INPUT_SIZE),
                        batch_size = 1)
def representative_dataset():
    for i in range(100):
        x, y = quant_set.next()
        yield [x]

# Convert the tflite.
converter = lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_quant_model = converter.convert()

# Save the model.
with open('trained.tflite', 'wb') as f:
  f.write(tflite_quant_model)

Found 2000 images belonging to 2 classes.


/usr/local/lib/python3.9/dist-packages/tensorflow/lite/python/convert.py:789: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


## Succeed

Copy trained.tflite to OpenMV4 H7 Plus, run this code in OpenMV4 H7 Plus: https://github.com/SingTown/openmv_tensorflow_training_scripts/blob/main/mobilenet_v2/main.py